In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import sys
sys.path.insert(1, '../src')
from src import PROJECT_PATHS

After this steps we will receive dataset with the following parameters:
1. Likes
2. Comments
3. Target
4. Sin hour / Cos hour
5. Sin month / Cos month
6. Sin weekend / Cos weekend
7. Embedded title 1-300
8. Embedded text 1-300

For embedding process the fasttext model was used. Lets do it step by step

## Some additional processing

In [3]:
df = pd.read_csv(f"{PROJECT_PATHS.data}\\processed\\preprocessed.csv", index_col=0)

### Split timestamp

In [4]:
df.drop(columns=['CURRENT_TIMESTAMP'], inplace=True)
df.TITLE = df.TITLE.str.lower()
df["TIMESTAMP"] = pd.to_datetime(df['TIMESTAMP'])
df.shape

(95580, 11)

In [5]:
df["HOUR"] = df["TIMESTAMP"].map(lambda x: x.hour)
df["YEAR"] = df["TIMESTAMP"].map(lambda x: x.year)
df["MONTH"] = df["TIMESTAMP"].map(lambda x: x.month)
df["WEEKDAY"] = df["TIMESTAMP"].map(lambda x: x.weekday())
df["WEEK"] = df["TIMESTAMP"].map(lambda x: x.week)

<ipython-input-5-9e8a95b6654e>:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df["WEEK"] = df["TIMESTAMP"].map(lambda x: x.week)


In [6]:
df.head()

,TITLE,LIKES,TEXT,TAGS,COMMENTS,NICKNAME,TIME_GAP,TIMESTAMP,TEXT_PUNCT,TITLE_PUNCT,target,HOUR,YEAR,MONTH,WEEKDAY,WEEK
0,мвд разработало правила отправки пьяных россия...,NaN,мвд россии разработало порядок доставления нет...,"['Полиция', 'Вытрезвитель', 'Опьянение', 'Прое...",50,Kanedias,2 месяца назад,2021-02-20 22:55:57+03:00,мвд россии разработало порядок доставления нет...,МВД разработало правила отправки пьяных россия...,7,22,2021,2,5,7
1,ответ на пост « геноцид 1933 года на страница...,-39.0,а что вы хотите национализм и шовинизм в том...,"['[моё]', 'Политика', 'Голод', 'Голодомор', 'У...",152,CultDeadCow,2 месяца назад,2021-02-20 18:44:21+03:00,"а что вы хотите? национализм и шовинизм, в том...","Ответ на пост «""Геноцид 1933 года"" на страница...",7,18,2021,2,5,7
2,сахалинский депутат лишился мандата из за виде...,186.0,депутаты томаринского городского округа юг са...,"['[моё]', 'Новости', 'Политика', 'Скандал', 'С...",30,TASSagency,2 месяца назад,2021-02-20 12:57:10+03:00,депутаты томаринского городского округа (юг са...,Сахалинский депутат лишился мандата из-за виде...,7,12,2021,2,5,7
3,рогозин отчитался путину об успехах «роскосмоса»,48.0,глава «роскосмоса» дмитрий рогозин на встрече ...,"['Дмитрий Рогозин', 'Новости', 'Политика', 'Ко...",33,ElonMuskSpaceX,2 месяца назад,2021-02-20 12:34:19+03:00,глава «роскосмоса» дмитрий рогозин на встрече ...,Рогозин отчитался Путину об успехах «Роскосмоса»,7,12,2021,2,5,7
4,москва ждет реакцию лондона на утечку данных о...,27.0,москва 20 февраля тасс рф ждет реакцию вел...,"['Мария Захарова', 'Политика', 'Великобритания...",57,rug178,2 месяца назад,2021-02-20 12:07:30+03:00,"москва, 20 февраля. /тасс/. рф ждет реакцию ве...",Москва ждет реакцию Лондона на утечку данных о...,7,12,2021,2,5,7


### Drop not used and fill na

In [7]:
df.drop(df[df['TITLE'] == 'удалено'].index, inplace=True)
df["TITLE"][df['TITLE'].isnull()] = ''
df['LIKES'] = df['LIKES'].fillna(0)
df.drop(columns=["TAGS", "NICKNAME", "TEXT_PUNCT", "TITLE_PUNCT", "YEAR", "WEEK", "TIMESTAMP", "TIME_GAP"], inplace=True)
df.drop(index = df.loc[~(df['TITLE'].str.len() > 0)].index, inplace=True)
print(df.shape)

(95580, 8)


<ipython-input-7-34c8cd3fe89e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TITLE"][df['TITLE'].isnull()] = ''


In [8]:
df.head()

,TITLE,LIKES,TEXT,COMMENTS,target,HOUR,MONTH,WEEKDAY
0,мвд разработало правила отправки пьяных россия...,0.0,мвд россии разработало порядок доставления нет...,50,7,22,2,5
1,ответ на пост « геноцид 1933 года на страница...,-39.0,а что вы хотите национализм и шовинизм в том...,152,7,18,2,5
2,сахалинский депутат лишился мандата из за виде...,186.0,депутаты томаринского городского округа юг са...,30,7,12,2,5
3,рогозин отчитался путину об успехах «роскосмоса»,48.0,глава «роскосмоса» дмитрий рогозин на встрече ...,33,7,12,2,5
4,москва ждет реакцию лондона на утечку данных о...,27.0,москва 20 февраля тасс рф ждет реакцию вел...,57,7,12,2,5


### Replace time with sin and cos

In [9]:
def make_cos_list(list_, period=24):
    def make_cos(value, period=period):
        return np.cos(value*2*np.pi/period)
    return [make_cos(x) for x in list_]

In [10]:
def make_sin_list(list_, period=24):
    def make_sin(value, period=period):
        return np.sin(value*2*np.pi/period)
    return [make_sin(x) for x in list_]

In [11]:
df['sin_hour'] = make_sin_list(df['HOUR'])
df['cos_hour'] = make_cos_list(df['HOUR'])

df['cos_month'] = make_cos_list(df['MONTH'], 12)
df['sin_month'] = make_sin_list(df["MONTH"], 12)

df['cos_weekday'] = make_cos_list(df['WEEKDAY'], 7) 
df['sin_weekday'] = make_sin_list(df['WEEKDAY'], 7)

df.drop(columns=['HOUR', 'MONTH', 'WEEKDAY'], inplace=True)

In [12]:
df.head()

,TITLE,LIKES,TEXT,COMMENTS,target,sin_hour,cos_hour,cos_month,sin_month,cos_weekday,sin_weekday
0,мвд разработало правила отправки пьяных россия...,0.0,мвд россии разработало порядок доставления нет...,50,7,-5.000000e-01,8.660254e-01,0.5,0.866025,-0.222521,-0.974928
1,ответ на пост « геноцид 1933 года на страница...,-39.0,а что вы хотите национализм и шовинизм в том...,152,7,-1.000000e+00,-1.836970e-16,0.5,0.866025,-0.222521,-0.974928
2,сахалинский депутат лишился мандата из за виде...,186.0,депутаты томаринского городского округа юг са...,30,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928
3,рогозин отчитался путину об успехах «роскосмоса»,48.0,глава «роскосмоса» дмитрий рогозин на встрече ...,33,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928
4,москва ждет реакцию лондона на утечку данных о...,27.0,москва 20 февраля тасс рф ждет реакцию вел...,57,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928


### Embedding (using fast text)

In [13]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stanislav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
from compress_fasttext.models import CompressedFastTextKeyedVectors

Download fasttext model (already downloaded)

In [15]:
import fasttext
# fasttext.util.download_model('ru', if_exists='ignore')  # English
# ft = fasttext.load_model('cc.ru.300.bin')

In [16]:
import pathlib
cur_path = pathlib.Path().absolute()
embs = f"{cur_path}\\cc.ru.300.bin"
embeddings = fasttext.load_model(embs)

In [17]:
def embed(tokens, default_size=300):
    if not tokens:
        return np.zeros(default_size)
    embs = [embeddings[_normalize_string(x)] for x in tokens]
    return sum(embs) / len(tokens)

def _normalize_string(text):
    return text.replace('ё', 'е')

def remove_stop_words(words_list):
    return [x for x in words_list if x not in sw]

In [18]:
def process_record(record):
    return embed(remove_stop_words(record.split()))

In [19]:
df['emb_title'] = df['TITLE'].map(process_record)

In [20]:
df['emb_text'] = df['TEXT'].map(str).map(process_record)

In [21]:
df.drop(columns=['TITLE', 'TEXT'], inplace=True)

In [22]:
df.head()

,LIKES,COMMENTS,target,sin_hour,cos_hour,cos_month,sin_month,cos_weekday,sin_weekday,emb_title,emb_text
0,0.0,50,7,-5.000000e-01,8.660254e-01,0.5,0.866025,-0.222521,-0.974928,"[0.06757413, -0.07326214, -0.018460793, -0.017...","[0.01676879, -0.025353305, 0.0008141762, 0.012..."
1,-39.0,152,7,-1.000000e+00,-1.836970e-16,0.5,0.866025,-0.222521,-0.974928,"[0.015744198, 0.0021398822, -0.008616511, 0.01...","[0.027725127, -0.020325447, -0.008841635, 0.02..."
2,186.0,30,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,"[0.054962788, -0.01747963, 0.032105397, 0.0041...","[0.027245743, -0.008372225, -0.007476216, 0.00..."
3,48.0,33,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,"[0.095593646, 0.026084613, -0.018681413, -0.03...","[0.037643507, -0.010600946, 0.008813703, -0.00..."
4,27.0,57,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,"[0.029514674, -0.035032023, -0.003657855, 0.00...","[0.033378504, -0.037415743, 0.0053046923, 0.00..."


In [23]:
df['emb_title'] = df['emb_title'].map(lambda x: json.dumps([float(y) for y in x]))
df['emb_text'] = df['emb_text'].map(lambda x: json.dumps([float(y) for y in x]))

In [24]:
df.isnull().sum()

LIKES          0
COMMENTS       0
target         0
sin_hour       0
cos_hour       0
cos_month      0
sin_month      0
cos_weekday    0
sin_weekday    0
emb_title      0
emb_text       0
dtype: int64

In [25]:
emb_text = pd.DataFrame(df['emb_text'].map(json.loads).to_list(),
                        columns=[f"emb_text_{i}" for i in range(300)])

emb_title = pd.DataFrame(np.array(df['emb_title'].map(json.loads).to_list()),
                         columns=[f"emb_title_{i}" for i in range(300)])

In [26]:
df_embedded = pd.concat([df.drop(columns=['emb_title', 'emb_text']).reset_index(drop=True),
                         emb_text, emb_title], axis=1)
df_embedded.head()

,LIKES,COMMENTS,target,sin_hour,cos_hour,cos_month,sin_month,cos_weekday,sin_weekday,emb_text_0,...,emb_title_290,emb_title_291,emb_title_292,emb_title_293,emb_title_294,emb_title_295,emb_title_296,emb_title_297,emb_title_298,emb_title_299
0,0.0,50,7,-5.000000e-01,8.660254e-01,0.5,0.866025,-0.222521,-0.974928,0.016769,...,-0.006696,0.007845,-0.022188,-0.030541,0.033095,0.005611,0.079799,-0.060192,0.025157,-0.014475
1,-39.0,152,7,-1.000000e+00,-1.836970e-16,0.5,0.866025,-0.222521,-0.974928,0.027725,...,0.009607,0.032843,-0.062798,-0.040111,-0.056778,-0.000490,-0.004771,0.044609,0.044650,0.067494
2,186.0,30,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,0.027246,...,-0.017774,0.024428,-0.010970,-0.017435,-0.016049,-0.002617,0.025836,0.001839,-0.010091,-0.012859
3,48.0,33,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,0.037644,...,0.003854,0.045091,0.015493,-0.021446,0.014062,-0.003290,0.072985,-0.022510,0.031925,0.009049
4,27.0,57,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,0.033379,...,-0.028516,0.010097,-0.012332,-0.029570,0.022079,-0.014889,0.041341,-0.025642,-0.012887,-0.030432


In [27]:
df_embedded.to_csv(f"{PROJECT_PATHS.data}\\processed\\final_embeded.csv")